# Chapter 1 - Prompt Chaining


In [3]:
# Imports
from langchain_openai import ChatOpenAI # Or import your preferred model like ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [7]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify if API key is present
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("OpenAI API key not found. Please check your .env file.")

# Initialize the OpenAI client and make the API call
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
response = client.chat.completions.create(
    model="gpt-4o-mini", temperature=0.7,
    messages=[
        {"role": "user", "content": "Hello OpenAI!, Write a 1 line small tip to highlight best software engineering practices!"}
    ]
)
# Print the response
print(response.choices[0].message.content)

Always prioritize clear documentation and code readability to enhance collaboration and maintainability in software projects.


## --- Initialize the language model ---

In [8]:

# Use the appropriate class and model name for your provider (e.g., ChatGoogleGenerativeAI(model="gemini-pro"))
# Setting temperature to control creativity (0.7 is a common balance)
try:
    # Example for OpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    # Example for Google (uncomment and replace if using Google)
    # from langchain_google_genai import ChatGoogleGenerativeAI
    # llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
    print(f"Language model initialized: {llm.model_name}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    print("Please ensure your API key is set correctly and the model name is valid.")
    llm = None # Set llm to None if initialization fails

Language model initialized: gpt-4o-mini


## --- Prompt Templates ---

In [9]:

# Define the first prompt template: Rephrase the question
# This template takes the original user question as input
rephrase_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at rephrasing questions to be clear, concise, and optimized for an AI assistant to answer."),
    ("user", "Rephrase the following question: {original_question}")
])

In [10]:
# Define the second prompt template: Answer the rephrased question
# This template takes the output from the rephrasing step as input
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Answer the following question based on your knowledge."),
    ("user", "{rephrased_question}")
])

## --- Building the Chain using LangChain Expression Language (LCEL) ---

In [12]:


# Step 1: The first part of the chain takes the original question,
# applies the rephrase_prompt, sends it to the LLM, and parses the output to a string.
rephrase_chain = rephrase_prompt | llm | StrOutputParser()
print(rephrase_chain)

first=ChatPromptTemplate(input_variables=['original_question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at rephrasing questions to be clear, concise, and optimized for an AI assistant to answer.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_question'], input_types={}, partial_variables={}, template='Rephrase the following question: {original_question}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12020c170>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1202206e0>, root_client=<openai.OpenAI object at 0x117ebfd90>, root_async_client=<openai.AsyncOpenAI object at 0x117e83770>, model_name='gpt-4o-mini', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********

In [13]:
# Step 2: Now, chain the output of the rephrase_chain to the answer_prompt
# The output of `rephrase_chain` (the rephrased question string) needs to be passed
# as the value for the `rephrased_question` variable in the `answer_prompt` template.
# LCEL allows us to do this mapping explicitly using a dictionary structure.
# The key in the dictionary ("rephrased_question") must match the input variable name
# expected by the next component (answer_prompt). The value is the preceding chain.
full_chain = {"rephrased_question": rephrase_chain} | answer_prompt | llm | StrOutputParser()
print(full_chain)

first={
  rephrased_question: ChatPromptTemplate(input_variables=['original_question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at rephrasing questions to be clear, concise, and optimized for an AI assistant to answer.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_question'], input_types={}, partial_variables={}, template='Rephrase the following question: {original_question}'), additional_kwargs={})])
                      | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12020c170>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1202206e0>, root_client=<openai.OpenAI object at 0x117ebfd90>, root_async_client=<openai.AsyncOpenAI object at 0x117e83770>, model_name='gpt-4o-mini', temperature=0.7, model_kwargs=

## --- Testing the Chain ---

In [19]:

if llm: # Only run if the model was initialized successfully
    original_question = "Tell me about the big statue in Paris people go to see?"

    print(f"Original question: {original_question}")
    print("-" * 30)
       

    try:
         # First, get the rephrased question
        rephrased = rephrase_chain.invoke({"original_question": original_question})
        print(f"Rephrased question: {rephrased}")
        print("-" * 30)

        # Invoke the chain with the initial input
        # Then get the final answer
        response = full_chain.invoke({"original_question": original_question})
        print(f"Final answer: {response}")

    except Exception as e:
        print(f"An error occurred during chain execution: {e}")
        print("Please check your API key, model name, and network connection.")
else:
    print("Chain execution skipped due to LLM initialization failure.")

Original question: Tell me about the big statue in Paris people go to see?
------------------------------
Rephrased question: What is the large statue in Paris that attracts many visitors?
------------------------------
------------------------------
Final answer: The large statue in Paris that attracts many visitors is the **Statue of Liberty** located on Liberty Island in the Seine River. It is a smaller replica of the original Statue of Liberty in New York City, which was a gift from France to the United States. The Paris statue was gifted by the French in 1889 and stands at approximately 11.5 meters (37 feet) tall, not including its pedestal. Visitors often come to see this iconic symbol of freedom and friendship between the two nations.


In [15]:
# --- Potential Extensions (Code Concepts - Explained in Text) ---
# - Adding Intermediate Logic: Insert Python functions using '|' in LCEL.
#   Example: rephrase_chain | some_python_function | {"rephrased_question": ...} | ...
# - Error Handling: Use .with_fallbacks() or try/except blocks around chain invocation.
# - State Management with LangGraph: For non-linear or stateful workflows,
#   LangGraph provides a more advanced canvas to define nodes and transitions.
#   This involves defining a graph state, nodes (LLM calls, functions, tools),
#   and edges (transitions, including conditional ones).
#   (Full LangGraph example is more complex and would be shown in a dedicated section or chapter)